In [6]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, ActivityRegularization, Dropout, BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from io import BytesIO
from PIL import Image


(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.01))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.01))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.01))
model.add(Dense(128, activation='relu'))
model.add(ActivityRegularization(l1=0.001, l2=0.001))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

model.fit(x_train, y_train, epochs=15, validation_data=(x_test, y_test))

class_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc}")


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 7200)              0         
                                                                 
 dense_25 (Dense)            (None, 1024)              7373824   
                                                                 
 dropout_15 (Dropout)        (None, 1024)              0         
                                                                 
 dense_26 (Dense)            (None, 1024)              1049600   
                                                      

UnidentifiedImageError: ignored

In [11]:
import requests
from io import BytesIO

image_url = 'https://i.natgeofe.com/n/548467d8-c5f1-4551-9f58-6817a8d2c45e/NationalGeographic_2572187_square.jpg'

# Download and preprocess the image from the URL
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))
img = img.resize((32, 32))  # Resize the image to match the model's input shape
img = tf.keras.preprocessing.image.img_to_array(img)
img = img / 255.0  # Normalize pixel values

prediction = model.predict(tf.convert_to_tensor([img]))
predicted_class = class_labels[prediction.argmax()]
print(predicted_class)

1/1 [==============================] - 0s 20ms/step
cat
